In [1]:
# Running %env without any arguments
# lists all environment variables

# The line below sets the environment
# variable CUDA_VISIBLE_DEVICES
%env CUDA_VISIBLE_DEVICES = 1

import numpy as np
import pandas as pd
import io
import time
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from scipy.misc import imread, imsave
import tensorflow as tf
from tensorflow.python.platform import tf_logging
import os.path
import tensorflow.contrib.slim as slim
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
import logging
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from densenet.nets import densenet
from densenet.preprocessing import densenet_preprocessing

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

env: CUDA_VISIBLE_DEVICES=1


In [2]:
DATASET_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/'
PRETRAINED_MODEL_PATH = DATASET_PATH + 'Dense/tf-densenet161.ckpt'
LOG_PATH = DATASET_PATH + 'Dense/logs-high/'
TRAIN_PATH = DATASET_PATH + 'Split1/Train/'
VAL_PATH = DATASET_PATH + 'Split1/Validation/'
TEST_PATH = DATASET_PATH + 'Test/'
CATEGORY_NAME_PATH = DATASET_PATH + 'category_names.csv'
BATCH_SIZE = 32
IMAGE_WIDTH = 180
IMAGE_HEIGHT = 180
NUM_CLASS = 5270
# validation examples num: 2319624
# train examples num: 10051704
# total step: 157057
TOTAL_EXAMPLES = 10051704
# validation num = 2319624
NUM_EPOCHES = 5
INPUT_THREADS = 6

#Learning rate information and configuration (Up to you to experiment)
initial_learning_rate = 0.00002#0.01
learning_rate_decay_factor = 0.88
num_epochs_per_decay = 0.15
moving_average_decay = 0.96
num_steps_per_epoch = TOTAL_EXAMPLES / BATCH_SIZE
decay_steps = int(TOTAL_EXAMPLES / BATCH_SIZE * num_epochs_per_decay)

SUB_NETWORK = 'densenet161'

In [3]:
# get TF logger
log = logging.getLogger('tensorflow')
log.setLevel(logging.DEBUG)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# create file handler which logs even debug messages
fh = logging.FileHandler(DATASET_PATH + 'tensorflow_densenet_train.log')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
log.addHandler(fh)

In [4]:
networks_collection = {  'densenet121': densenet.densenet121,
                         'densenet161': densenet.densenet161,
                         'densenet169': densenet.densenet169 }

In [5]:
class MiniDataSet(object):
    def __init__(self, file_path_pattern, category_level_csv, num_examples, num_classes, is_training = True, min_after_dequeue=1000, batch_size = BATCH_SIZE, num_epochs = NUM_EPOCHES, num_reader = INPUT_THREADS):
        super(MiniDataSet, self).__init__()
        self._num_examples = num_examples
        self._num_classes = num_classes
        self._file_path_pattern = file_path_pattern
        self._category_level_csv = category_level_csv
        self._num_reader = num_reader
        self._batch_size = batch_size
        self._num_epochs = num_epochs
        self._min_after_dequeue = min_after_dequeue
        self._is_training = is_training
        
    def get_category_description_from_csv(self, level = 0):
        category_map = dict()
        csv = pd.read_csv(self._category_level_csv).values
        for row in csv:  
            category_id, levels = row[0], row[1:]
            category_map[category_id] = levels[level]
        return category_map

    def create_dataset(self):
        opts = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.ZLIB)
        reader = lambda : tf.TFRecordReader(options=opts)
        keys_to_features = {
            'img_raw': tf.FixedLenFeature([], tf.string, default_value=''),
            'product_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
            # notice that we don't have this feature in our TFRecord, so always default provided
            'format': tf.FixedLenFeature([], tf.string, default_value='jpg'),
            'category_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))
        }

        items_to_handlers = {
            # automated decode image from features in FixedLenFeature
            'image': slim.tfexample_decoder.Image(image_key='img_raw', format_key='format'),
            'label': slim.tfexample_decoder.Tensor('category_id'),
        }

        decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)

        labels_to_name_dict = self.get_category_description_from_csv()

        self._dataset = slim.dataset.Dataset(
            data_sources = self._file_path_pattern,
            decoder = decoder,
            reader = reader,
            # num_readers = 8,
            num_samples = self._num_examples,
            #num_classes = self._num_classes,
            #labels_to_name = labels_to_name_dict,
            items_to_descriptions = None)
        
        # notice that DatasetDataProvider can automate shuffle the examples by ParallelReader using its RandomShuffleQueue
        self._data_provider = slim.dataset_data_provider.DatasetDataProvider(
            self._dataset,
            num_readers = self._num_reader,
            shuffle = True, # default is True
            num_epochs = self._num_epochs,
            common_queue_capacity = self._min_after_dequeue + 3 * self._batch_size,
            common_queue_min = self._min_after_dequeue,
            scope = self._is_training and 'train_files' or 'validation_files')
        
        return self._data_provider.get(['image', 'label'])
        

In [6]:
default_image_size = networks_collection[SUB_NETWORK].default_image_size
def preprocess_for_densenet(input_image, is_training = True):
    # densenet.default_image_size = 224
    return densenet_preprocessing.preprocess_image(input_image, default_image_size, default_image_size, is_training=is_training)

In [7]:
def cvt_csv2tfrecord():
    count = 0
    category_map = dict()
    csv = pd.read_csv(CATEGORY_NAME_PATH).values
    for row in csv:  
        category_id, _ = row[0], row[1:]
        category_map[category_id] = count
        count += 1
    return category_map

In [8]:
def one_hot_process(org_label, map_table, num_classes):
    return tf.one_hot(map_table.lookup(tf.as_string(org_label)), num_classes, axis=-1)

In [9]:
def_graph = tf.Graph()
with def_graph.as_default() as graph:
    def train_step(input_examples, one_hot_labels):   
         with slim.arg_scope(densenet.densenet_arg_scope(weight_decay=0.00004)):
            logits, end_points = networks_collection[SUB_NETWORK](input_examples, NUM_CLASS, is_training=True)
            logits = tf.squeeze(logits)

            # we retrain for diferrent num classes
            # and don't define any Variables before get_variables_to_restore
            variables_to_restore = slim.get_variables_to_restore(exclude = ['global_step', 'densenet121/logits', 'densenet161/logits/weights', 'densenet161/logits/biases'])
            # Performs the equivalent to tf.nn.sparse_softmax_cross_entropy_with_logits but enhanced, e.x. label smothing
            loss = tf.losses.softmax_cross_entropy(onehot_labels = one_hot_labels, logits = logits)
            total_loss = tf.losses.get_total_loss()    # obtain the regularization losses as well

            # Create the global step for monitoring the learning_rate and training.
            # since supervisor will also create one global_step, so we create n advance in order to feed into exponential_decay
            global_step = get_or_create_global_step(graph = graph)

            #Define your exponentially decaying learning rate
            lr = tf.train.exponential_decay(
                learning_rate = initial_learning_rate,
                global_step = global_step,
                decay_steps = decay_steps,
                decay_rate = learning_rate_decay_factor,
                staircase = True)

            #Now we can define the optimizer that takes on the learning rate
            optimizer = tf.train.AdamOptimizer(learning_rate = lr)

            # Gather update_ops from the first clone. These contain, for example,
            # the updates for the batch_norm variables created by network_fn.
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

            moving_average_variables = slim.get_model_variables()
            variable_averages = tf.train.ExponentialMovingAverage(moving_average_decay, global_step)
            # Use an alternative set of update ops in addition to the default updates:
            tf.add_to_collection(tf.GraphKeys.UPDATE_OPS, variable_averages.apply(moving_average_variables))

            #Create the train_op.
            train_op = slim.learning.create_train_op(total_loss, optimizer, summarize_gradients=False)

            #State the metrics that you want to predict. We get a predictions that is not one_hot_encoded.
            #predictions = end_points['predictions']
            predictions = tf.argmax(tf.squeeze(end_points['predictions']), 1)
            probabilities = end_points['predictions']
            accuracy, accuracy_update = slim.metrics.streaming_accuracy(predictions, tf.argmax(one_hot_labels, 1))
            metrics_op = tf.group(accuracy_update)


            #Now finally create all the summaries you need to monitor and group them into one summary op.
            tf.summary.scalar('losses/Total_Loss', total_loss)
            tf.summary.scalar('accuracy', accuracy)
            tf.summary.scalar('learning_rate', lr)

            return train_op, global_step, metrics_op, variables_to_restore, predictions, lr, accuracy, total_loss

    def validation_step(input_examples, one_hot_labels):   
        with slim.arg_scope(densenet.densenet_arg_scope(weight_decay=0.00004)):
            logits, end_points = networks_collection[SUB_NETWORK](input_examples, NUM_CLASS, is_training=False, reuse = True)
            logits = tf.squeeze(logits)

            #State the metrics that you want to predict. We get a predictions that is not one_hot_encoded.
            predictions = tf.argmax(tf.squeeze(end_points['predictions']), 1)
            
            probabilities = end_points['predictions']
            #tf.squeeze(predictions)  tf.reshape(predictions, [-1, NUM_CLASS])
            accuracy, accuracy_update = slim.metrics.streaming_accuracy(predictions, tf.argmax(one_hot_labels, 1))
            metrics_op = tf.group(accuracy_update)

            #Now finally create all the summaries you need to monitor and group them into one summary op.
            tf.summary.scalar('validation/accuracy', accuracy)

            return metrics_op, accuracy, predictions, probabilities

In [ ]:
with def_graph.as_default() as graph:
    def init_dataset(file_path_pattern, mapping_table, is_training = True):
        dataset = MiniDataSet(file_path_pattern, CATEGORY_NAME_PATH, TOTAL_EXAMPLES, NUM_CLASS, is_training = is_training)
        org_image, org_label = dataset.create_dataset()
        image = preprocess_for_densenet(org_image, is_training) # final image to train

        label = one_hot_process(org_label, mapping_table, NUM_CLASS) # final label for training
        # no need for shuffle, DatasetDataProvider do this for us
        batch_images, batch_labels = tf.train.batch([image, label], BATCH_SIZE,\
                                            num_threads = INPUT_THREADS,\
                                            capacity = 1000 + 3 * BATCH_SIZE,\
                                            allow_smaller_final_batch = is_training, name = is_training and 'train_batch' or 'validation_batch')
        
        return batch_images, batch_labels

In [ ]:
with def_graph.as_default() as graph:
    mapping_strings = tf.constant( [ str(key) for key in cvt_csv2tfrecord().keys() ] )
    mapping_table = tf.contrib.lookup.index_table_from_tensor(mapping=mapping_strings, default_value=0)
    batch_images, batch_labels = init_dataset(TRAIN_PATH + "output_file*.tfrecords", mapping_table)
    batch_val_images, batch_val_labels = init_dataset(VAL_PATH + "test_output_file*.tfrecords", mapping_table, False)
    with tf.device('/gpu:0'):
        train_op, global_step, metrics_op, variables_to_restore, pred_op, lr, accuracy, total_loss = train_step(batch_images, batch_labels)
        val_metrics_op, val_accuracy, val_predictions, val_probabilities = validation_step(batch_val_images, batch_val_labels)
        real_val_label = tf.argmax(batch_val_labels, 1)
                    
    summary_op = tf.summary.merge_all()
    # Create a saver that restores only the pre-trained variables.
    pre_train_saver = tf.train.Saver(variables_to_restore)
    # Define an init function that loads the pretrained checkpoint.
    # sess is the managed session passed by Supervisor
    def load_pretrain(sess):
        pre_train_saver.restore(sess, PRETRAINED_MODEL_PATH)

    # no need for specify local_variables_initializer and tables_initializer, Supervisor will do this via default local_init_op
    # init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer(), tf.tables_initializer())
    init_op = tf.group(tf.global_variables_initializer())
    # Pass the init function to the supervisor.
    # - The init function is called _after_ the variables have been initialized by running the init_op.
    # - use default tf.Saver() for ordinary save and restore
    # - save checkpoint every 1.3 hours(4800)
    # - manage summary in current process by ourselves for memory saving
    # - no need to specify global_step, supervisor will find this automately
    # - initialize order: checkpoint -> local_init_op -> init_op -> init_func
    sv = tf.train.Supervisor(logdir=LOG_PATH, init_fn = load_pretrain, init_op = init_op, summary_op = None, save_model_secs=24000, checkpoint_basename='densenet_model.ckpt')
    
    final_loss = 0.
    final_accuracy = 0.
    training_state = True
    with sv.managed_session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)) as sess:
    #with sv.prepare_or_wait_for_session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)) as sess:

        # Here sess was either initialized from the pre-trained-checkpoint or
        # recovered from a checkpoint saved in a previous run of this code.
        for step in range(int(num_steps_per_epoch * NUM_EPOCHES)):       
            if sv.should_stop():
                tf_logging.info('Supervisor emit finished!')
                tf_logging.info('Current Loss: %s', loss)
                tf_logging.info('Current Accuracy: %s', accuracy)
                tf_logging.info('Saving current model to disk(maybe invalid).')
                training_state = False
                break

            start_time = time.time()
            if step % 500 == 0:
                with tf.device('/gpu:0'):
                    _, _, _, summ = sess.run([train_op, global_step, metrics_op, summary_op])
                sv.summary_computed(sess, summ)
            else:
                if step % 50 == 0:
                    with tf.device('/gpu:0'):
                        _, val_acc, val_pred, val_prob, real_label = sess.run([val_metrics_op, val_accuracy, val_predictions, val_probabilities, real_val_label])
                    time_elapsed = time.time() - start_time
                    tf_logging.info('Validation Speed: {:5.3f}sec/batch'.format(time_elapsed))
                    tf_logging.info('Current Streaming ValAccuracy: {:5.3f}%'.format(val_acc*100.))
                    tf_logging.info('Real Label: {}'.format(real_label))
                    tf_logging.info('Pred Label: {}'.format(val_pred))
                        
                else:
                    with tf.device('/gpu:0'):
                        _, total_step, _, cur_loss, cur_acc, cur_lr = sess.run([train_op, global_step, metrics_op, total_loss, accuracy, lr])
                    time_elapsed = time.time() - start_time
                    if step % 10 == 0:
                        final_loss = cur_loss
                        final_accuracy = cur_acc
                        tf_logging.info('Current Speed: {:5.3f}sec/batch'.format(time_elapsed))
                        tf_logging.info('Current Streaming Accuracy: {:5.3f}%'.format(cur_acc*100.))
                        tf_logging.info('Current Loss: {:5.3f}'.format(cur_loss))
                        tf_logging.info('Epoch %s/%s, Global Step: %s', int(total_step / num_steps_per_epoch + 1), NUM_EPOCHES, total_step)
                        tf_logging.info('Current Learning Rate: {}'.format(cur_lr))
                
                    
        if training_state:
            #We log the final training loss and accuracy
            tf_logging.info('Final Loss: %s', final_loss)
            tf_logging.info('Final Accuracy: %s', final_accuracy)
            # Once all the training has been done, save the log files and checkpoint model
            tf_logging.info('Finished training! Model saved.')
        sv.saver.save(sess, sv.save_path, global_step = sv.global_step)
    

INFO:tensorflow:Restoring parameters from /media/rs/0E06CD1706CD0127/Kapok/kaggle/Dense/logs-high/densenet_model.ckpt-351980
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path /media/rs/0E06CD1706CD0127/Kapok/kaggle/Dense/logs-high/densenet_model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Current Speed: 0.792sec/batch
INFO:tensorflow:Current Streaming Accuracy: 66.562%
INFO:tensorflow:Current Loss: 3.052
INFO:tensorflow:Epoch 2/5, Global Step: 352000
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.801sec/batch
INFO:tensorflow:Current Streaming Accuracy: 65.000%
INFO:tensorflow:Current Loss: 3.707
INFO:tensorflow:Epoch 2/5, Global Step: 352010
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.726sec/batch
INFO:tensorflow:Current Streaming Accuracy: 61.875%
INFO:tensorflow:Current Loss: 3.806
INFO:tensorflow:Epoc

INFO:tensorflow:Current Streaming Accuracy: 56.534%
INFO:tensorflow:Current Loss: 5.197
INFO:tensorflow:Epoch 2/5, Global Step: 352265
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:global_step/sec: 1.275
INFO:tensorflow:Current Speed: 0.827sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.491%
INFO:tensorflow:Current Loss: 4.027
INFO:tensorflow:Epoch 2/5, Global Step: 352275
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Validation Speed: 0.170sec/batch
INFO:tensorflow:Current Streaming ValAccuracy: 70.625%
INFO:tensorflow:Real Label: [4815 3692  991 1005 4459 1122 4393 4987  377  178 2023 4045 4153  689 3728
  101 4861 3300 3797 1539  289 4564 2285 4816 4350 4236 3641 2008  255 1377
 2887 2938]
INFO:tensorflow:Pred Label: [4815 3692   57 1005 4708 1122  202 4987  377  178  536 4045 4153  689 3728
  101 4861 3300 3797 1106  289 4564 2285 4816 4350  689 1966 2008  255 1377
 2887 2938]
INFO:tensorflow:Current Speed: 0.85

INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.797sec/batch
INFO:tensorflow:Current Streaming Accuracy: 57.516%
INFO:tensorflow:Current Loss: 3.767
INFO:tensorflow:Epoch 2/5, Global Step: 352560
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.757sec/batch
INFO:tensorflow:Current Streaming Accuracy: 57.489%
INFO:tensorflow:Current Loss: 4.660
INFO:tensorflow:Epoch 2/5, Global Step: 352570
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Validation Speed: 0.180sec/batch
INFO:tensorflow:Current Streaming ValAccuracy: 71.250%
INFO:tensorflow:Real Label: [4325 3773 1277  232 4393 3692  457 1386 2681 3692 4350  398 3748 3623 4160
 3756 3692 4957 2886   73 2166 3593  289 3879 4045 3463 4045 3571  457 4045
 3330 4153]
INFO:tensorflow:Pred Label: [1162 3773 1277  232 3692 3692  457 1386 2819 3692 4350  398 3748 3623 4160
 3556 3692  762 2886   74 1551 3593  289 1892 4045 3463

INFO:tensorflow:Pred Label: [4194 3623 4160 3643 3597  289 3569  372 3165 2938 3501  138 4350 2865 2841
 5001 4236 3692  807 4671 5039 5241 1836 4970 4429 5130 2764  377 1301 3877
 3277  643]
INFO:tensorflow:Current Speed: 0.754sec/batch
INFO:tensorflow:Current Streaming Accuracy: 57.005%
INFO:tensorflow:Current Loss: 3.985
INFO:tensorflow:Epoch 2/5, Global Step: 352834
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.728sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.971%
INFO:tensorflow:Current Loss: 3.720
INFO:tensorflow:Epoch 2/5, Global Step: 352844
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.803sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.923%
INFO:tensorflow:Current Loss: 4.607
INFO:tensorflow:Epoch 2/5, Global Step: 352854
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.750sec/batch
INFO:tensorflow:Current Streaming Ac

INFO:tensorflow:Pred Label: [4045 3583 4800 1642 3689 2421 2159 3262  289 1282 4814 1245 3605 4957 4957
  289 1301 4350 4350 2886 1005 1301 3623  991 4350 4350 1277  453 1435 4045
 4045 2008]
INFO:tensorflow:Current Speed: 0.741sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.763%
INFO:tensorflow:Current Loss: 4.258
INFO:tensorflow:Epoch 2/5, Global Step: 353129
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.790sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.737%
INFO:tensorflow:Current Loss: 4.104
INFO:tensorflow:Epoch 2/5, Global Step: 353139
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.802sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.671%
INFO:tensorflow:Current Loss: 4.919
INFO:tensorflow:Epoch 2/5, Global Step: 353149
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.794sec/batch
INFO:tensorflow:Current Streaming Ac

...

INFO:tensorflow:Validation Speed: 0.198sec/batch
INFO:tensorflow:Current Streaming ValAccuracy: 65.972%
INFO:tensorflow:Real Label: [4487 4172  453 2166 4115 3671 4153 4815 4560 4279 2841 4141  662 2531 3593
 4814 4429  212 1343 1706 3456 3929 1930 3623 3102 4970 1954 2145 1343 4403
 3623 4305]
INFO:tensorflow:Pred Label: [4487 2791 2902 2177 4115 3671 4153 4815 3891 4279 2841 5130  606 4749 3593
 4814 4429  212 1343 1706 3456 3929 4248 3623 4314 4970  517  173 1343 4403
 3623 3279]
INFO:tensorflow:Current Speed: 0.845sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.607%
INFO:tensorflow:Current Loss: 4.637
INFO:tensorflow:Epoch 2/5, Global Step: 357450
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.932sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.601%
INFO:tensorflow:Current Loss: 3.911
INFO:tensorflow:Epoch 2/5, Global Step: 357460
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Spee

INFO:tensorflow:Current Speed: 0.881sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.586%
INFO:tensorflow:Current Loss: 3.841
INFO:tensorflow:Epoch 2/5, Global Step: 357715
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.917sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.596%
INFO:tensorflow:Current Loss: 4.384
INFO:tensorflow:Epoch 2/5, Global Step: 357725
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Validation Speed: 0.178sec/batch
INFO:tensorflow:Current Streaming ValAccuracy: 65.863%
INFO:tensorflow:Real Label: [3795 4662  174 3501 1500 5089 4335 3741  923 2840  689 2902 2821 2878 4350
  593   58 3692 3188 4809 3762 5054  114 4839 1648  287 3277 3050 3328 3226
 4160 2649]
INFO:tensorflow:Pred Label: [3795 3411  174 3501 1500 5089 4335 3741  923 2842  689 2902 2821 2878 4350
 1712   58 3692 3188 1245 3355 5054  114 4839 1645 4254 3277 3038 3328 3141
 4160 1625]
INFO:tensorflow:Current Spee

INFO:tensorflow:Current Speed: 0.872sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.613%
INFO:tensorflow:Current Loss: 4.261
INFO:tensorflow:Epoch 2/5, Global Step: 358010
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.850sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.620%
INFO:tensorflow:Current Loss: 3.668
INFO:tensorflow:Epoch 2/5, Global Step: 358020
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Validation Speed: 0.175sec/batch
INFO:tensorflow:Current Streaming ValAccuracy: 66.108%
INFO:tensorflow:Real Label: [2159 4160 2816 3823 3330 5210 2849 4548 4429 2842 1277  337 2908 3277 4861
 4429 1345  688 2781 4837 1736 1607  457 4403 1435 3741 3262  232 1644 2331
 4429 3184]
INFO:tensorflow:Pred Label: [5032 4160 2816 3823 3330 5210 2841 4562 4429 2842 1277  144 2908 3277 4861
 4429 3272 2738 2781 4837 1736 1607  457 4814 1435 3414 3262  232 1644 3441
 4429 3186]
INFO:tensorflow:Current Spee

INFO:tensorflow:Current Speed: 0.917sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.650%
INFO:tensorflow:Current Loss: 3.755
INFO:tensorflow:Epoch 2/5, Global Step: 358284
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.907sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.650%
INFO:tensorflow:Current Loss: 3.863
INFO:tensorflow:Epoch 2/5, Global Step: 358294
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.839sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.641%
INFO:tensorflow:Current Loss: 4.010
INFO:tensorflow:Epoch 2/5, Global Step: 358304
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.918sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.644%
INFO:tensorflow:Current Loss: 4.763
INFO:tensorflow:Epoch 2/5, Global Step: 358314
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Validation Speed

INFO:tensorflow:Current Speed: 0.958sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.569%
INFO:tensorflow:Current Loss: 3.913
INFO:tensorflow:Epoch 2/5, Global Step: 358579
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.877sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.573%
INFO:tensorflow:Current Loss: 4.176
INFO:tensorflow:Epoch 2/5, Global Step: 358589
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.927sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.582%
INFO:tensorflow:Current Loss: 4.286
INFO:tensorflow:Epoch 2/5, Global Step: 358599
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Current Speed: 0.898sec/batch
INFO:tensorflow:Current Streaming Accuracy: 56.591%
INFO:tensorflow:Current Loss: 3.755
INFO:tensorflow:Epoch 2/5, Global Step: 358609
INFO:tensorflow:Current Learning Rate: 8.173511560016777e-06
INFO:tensorflow:Validation Speed